In [8]:
from qiskit import *
import numpy as np

def Hamiltonian(n,h):
    pow_n=2**n
    qc = np.empty(2*n-1, dtype=object)
    #Creating the quantum circuits that are used in the calculation of the Hamiltonian based on the number of qubits
    for i in range(0, 2*n-1): #2n-1 is the number of factors on the n-site Hamiltonian
        qr = QuantumRegister(n) 
        qc[i] = QuantumCircuit(qr) #create quantum circuits for each factor of the Hamiltonian
        #print(i)
        if (i<=n-2): #for the first sum of the Hamiltonian
            qc[i].z(i) #value of current spin
            qc[i].z(i+1) #and value of its neighboring spin
        else: #for the second sum of the Hamiltonian
            qc[i].x(2*n-2-i) #2*n-2 gives the proper index since counting starts at 0
    #Run each circuit in the simulator        
    simulator = Aer.get_backend('unitary_simulator')
    result = np.empty(2*n-1, dtype=object) 
    unitary = np.empty(2*n-1, dtype=object) 
    Hamiltonian_Matrix=0
    #Get the results for each circuit in unitary form
    for i in range(0, 2*n-1):
        result[i] = execute(qc[i], backend=simulator).result()
        unitary[i] = result[i].get_unitary()
        #print(unitary[i])
        #And calculate the Hamiltonian matrix according to the formula
        if (i<=n-2):
            Hamiltonian_Matrix=np.add(Hamiltonian_Matrix,-unitary[i])
        else:
            Hamiltonian_Matrix=np.add(Hamiltonian_Matrix,-h*unitary[i])
    print("The",pow_n,"x",pow_n, "Hamiltonian Matrix is:")
    print(Hamiltonian_Matrix)
    #Now that we have the Hamiltonian
    
    #find the eigenvalues and eigenvectors
    w, v = np.linalg.eig(Hamiltonian_Matrix)
    #first column of the eigenvectors is the groundstate of the system
    groundstate = v[:,1]
    #the probability to measure each basic state of n qubits
    probability = np.square(groundstate).real
    print("The probability for each of the",pow_n,"base states is:")
    print(probability)
    print("The probabilities for each of the",pow_n,"base states add up to:")
    print ("%.2f" % np.sum(probability))
    
Hamiltonian(8,1)

The 256 x 256 Hamiltonian Matrix is:
[[-7.+0.j -1.+0.j -1.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j -5.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [-1.+0.j  0.+0.j -3.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 ...
 [ 0.+0.j  0.+0.j  0.+0.j ... -3.+0.j  0.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j -5.+0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ... -1.+0.j -1.+0.j -7.+0.j]]
The probability for each of the 256 base states is:
[7.84038200e-05 1.78451623e-04 4.58280633e-04 1.47139886e-04
 3.81636830e-04 1.34983763e-03 3.01971042e-04 1.65188284e-04
 4.09860376e-04 6.95935265e-04 3.64323118e-03 9.92028425e-04
 2.58034877e-04 7.42358978e-04 3.78770690e-04 1.50640592e-04
 4.09860376e-04 1.11222791e-03 1.76997600e-03 6.61296368e-04
 3.14546701e-03 1.19388990e-02 2.01615143e-03 1.26499018e-03
 2.80260557e-04 5.68226706e-04 2.03401400e-03 6.13060204e-04
 3.34063822e-04 1.12244196e-03 3.36901937e-04 1.65188284e-04
 3.81636830e-04 7.56852012e-04 2.68581278e-03 7.85457622e-04
 1.32062540e-03 4.29832738

In [9]:
#To test the algorithm, we should run the Hamiltonian for the two extreme cases where h=0 and h>>1, h being the intensity of the transverse field (check readme for math details)

#for h=0, the transverse field is completely gone and therefore there is 100% chance that we measure the spin were all spins point down (basis state)
print("The Hamiltonian for h=0 is")
Hamiltonian(8,0)
#as described by the results, the probability matrix has 0 all its elements, except one

The Hamiltonian for h=0 is
The 256 x 256 Hamiltonian Matrix is:
[[-7.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -5.+0.j  0.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -3.+0.j ...  0.+0.j  0.+0.j  0.+0.j]
 ...
 [ 0.+0.j  0.+0.j  0.+0.j ... -3.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j -5.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j ...  0.+0.j  0.+0.j -7.+0.j]]
The probability for each of the 256 base states is:
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

In [10]:
#for h=100 the transverse field is strong enough to push every spin into equal superposition, meaning that there is an equal probability to measure the system in each one of the basic states
print("The Hamiltonian for h=100 is")
Hamiltonian(8,100)
#as described by the results, the probability matrix elements are very close in value

The Hamiltonian for h=100 is
The 256 x 256 Hamiltonian Matrix is:
[[  -7.+0.j -100.+0.j -100.+0.j ...    0.+0.j    0.+0.j    0.+0.j]
 [-100.+0.j   -5.+0.j    0.+0.j ...    0.+0.j    0.+0.j    0.+0.j]
 [-100.+0.j    0.+0.j   -3.+0.j ...    0.+0.j    0.+0.j    0.+0.j]
 ...
 [   0.+0.j    0.+0.j    0.+0.j ...   -3.+0.j    0.+0.j -100.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j ...    0.+0.j   -5.+0.j -100.+0.j]
 [   0.+0.j    0.+0.j    0.+0.j ... -100.+0.j -100.+0.j   -7.+0.j]]
The probability for each of the 256 base states is:
[0.00377185 0.00380966 0.00384795 0.00380957 0.00384785 0.00388662
 0.00384776 0.00380957 0.00384785 0.00388643 0.00392568 0.00388652
 0.00384766 0.00388643 0.00384776 0.00380957 0.00384785 0.00388643
 0.00392549 0.00388633 0.00392558 0.00396514 0.00392549 0.00388652
 0.00384766 0.00388623 0.00392549 0.00388633 0.00384766 0.00388643
 0.00384776 0.00380957 0.00384785 0.00388643 0.00392549 0.00388633
 0.00392539 0.00396494 0.00392529 0.00388633 0.00392558 0.00396494
 0.00